# RNN序列编码-分类期末大作业

本次大作业要求手动实现双向LSTM+基于attention的聚合模型，并用于古诗作者预测的序列分类任务。**请先阅读ppt中的作业说明。**

In [334]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from numba import jit

import random
import numpy as np

from tqdm import tqdm

device = torch.device("cuda")

random.seed(1)
np.random.seed(1)
torch.manual_seed(1)

## 1. 加载数据

数据位于`data`文件夹中，每一行对应一个样例，格式为“诗句 作者”。下面的代码将数据文件读取到`train_data`, `valid_data`和`test_data`中，并根据训练集中的数据构造词表`word2idx`/`idx2word`和标签集合`label2idx`/`idx2label`。

In [335]:
word2idx = {"<unk>": 0}
label2idx = {}
idx2word = ["<unk>"]
idx2label = []

train_data = []
with open("./data/train.txt", encoding='utf-8') as f:
    for line in f:
        text, author = line.strip().split()
        for c in text:
            if c not in word2idx:
                word2idx[c] = len(idx2word)
                idx2word.append(c)
        if author not in label2idx:
            label2idx[author] = len(idx2label)
            idx2label.append(author)
        train_data.append((text, author))

valid_data = []
with open("./data/valid.txt", encoding='utf-8') as f:
    for line in f:
        text, author = line.strip().split()
        valid_data.append((text, author))

test_data = []
with open("./data/test.txt", encoding='utf-8') as f:
    for line in f:
        text, author = line.strip().split()
        test_data.append((text, author))

In [336]:
print(len(word2idx), len(idx2word), len(label2idx), len(idx2label))
print(len(train_data), len(valid_data), len(test_data))

4941 4941 5 5
11271 1408 1410


**请完成下面的函数，其功能为给定一句古诗和一个作者，构造RNN的输入。** 这里需要用到上面构造的词表和标签集合，对于不在词表中的字用\<unk\>代替。

In [337]:
def make_data(text, author):
    """
    输入
        text: str
        author: str
    输出
        x: LongTensor, shape = (1, text_length)
        y: LongTensor, shape = (1,)
    """
    # 构建词表
    x = torch.zeros(len(text), dtype=torch.long)
    for c in range(len(text)):
        if text[c] in word2idx:
            x[c] = word2idx[text[c]]
        else:
            x[c] = word2idx['<unk>']
    # 构建标签
    if author in label2idx:
        y = torch.tensor([label2idx[author]], dtype=torch.long)
    else:
        y = torch.tensor([label2idx['<unk>']], dtype=torch.long)
    return x, y

In [338]:
print(text, author, len(text))
print(make_data(text, author))

雲台高議正紛紛，誰定當時蕩寇勳。 李商隱 16
(tensor([ 182,  237,  109, 1898,  820, 1103, 1103,    8,  181,  377,  799,   32,
         817, 2241, 2274,   16]), tensor([0]))


## 2. LSTM算子（单个时间片作为输入）

In [339]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LSTM, self).__init__()
        self.f = nn.Linear(input_size + hidden_size, hidden_size)
        self.i = nn.Linear(input_size + hidden_size, hidden_size)
        self.o = nn.Linear(input_size + hidden_size, hidden_size)
        self.g = nn.Linear(input_size + hidden_size, hidden_size)
    
    def forward(self, ht, ct, xt):
        # ht: 1 * hidden_size
        # ct: 1 * hidden_size
        # xt: 1 * input_size
        input_combined = torch.cat((xt, ht), 1)
        ft = torch.sigmoid(self.f(input_combined))
        it = torch.sigmoid(self.i(input_combined))
        ot = torch.sigmoid(self.o(input_combined))
        gt = torch.tanh(self.g(input_combined))
        ct = ft * ct + it * gt
        ht = ot * torch.tanh(ct)
        return ht, ct

## 3. 实现双向LSTM（整个序列作为输入）

**要求使用上面提供的LSTM算子，不要调用torch.nn.LSTM**

In [340]:
class BiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(BiLSTM, self).__init__()
        # TODO
        self.lstm_forward = LSTM(input_size, hidden_size)
        self.lstm_backward = LSTM(input_size, hidden_size)
        self.hidden_size = hidden_size # 隐藏层维度
        self.input_size = input_size   # 输入（embedding）维度
        self.register_buffer("_float", torch.zeros(1, hidden_size))
    
    def init_h_and_c(self):
        h = torch.zeros_like(self._float).to(device)
        c = torch.zeros_like(self._float).to(device)
        return h, c
    
    def forward(self, x):
        """
        输入
            x: 1 * length * input_size
        输出
            hiddens
        """
        # TODO
        length = x.shape[0]
        hiddens_forward, hiddens_backward, hiddens = [], [], []

        # 从前往后的LSTM
        hidden_forward, cell_forward = self.init_h_and_c()
        for i in range(length):
            hidden_forward, cell_forward = self.lstm_forward(hidden_forward, cell_forward, x[i,:].unsqueeze(0))
            hiddens_forward.append(hidden_forward)

        hiddens_forward = torch.stack(hiddens_forward, dim=0).squeeze(1)
        
        # 从后往前的LSTM
        hidden_backward = torch.zeros(self.hidden_size).unsqueeze(0).to(device)
        cell_backward = torch.zeros(self.hidden_size).unsqueeze(0).to(device)
        for i in range(length-1, -1, -1):
            hidden_backward, cell_backward = self.lstm_backward(hidden_backward, cell_backward, x[i,:].unsqueeze(0))
            hiddens_backward.append(hidden_backward)
        
        hiddens_backward = torch.stack(hiddens_backward, dim=0).squeeze(1)
        hiddens_backward = torch.flip(hiddens_backward, dims=[0])

        # 将两个hidden向量拼接起来
        hiddens = torch.cat((hiddens_forward, hiddens_backward), -1).unsqueeze(0)

        return hiddens

## 4. 实现基于attention的聚合机制

In [341]:
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        # TODO
        self.q = nn.Linear(hidden_size, 1, bias=False) # attention层
        self.softmax = nn.Softmax(dim=0)
        self.hidden_size = hidden_size
    
    def forward(self, hiddens):
        """
        输入
            hiddens: 1 * length * hidden_size
        输出
            attn_outputs: 1 * hidden_size
        """
        # TODO
        # 计算每个时间片隐藏状态向量的权重
        a = self.q(hiddens.squeeze(0)).reshape(-1)
        alpha = self.softmax(a)

        # 加权求和得到经过q向量attention后的隐藏状态
        attn_outputs = torch.mm(hiddens[0].T, alpha.unsqueeze(1))
        
        #print("Attention: ", attn_outputs.shape)
        return attn_outputs.reshape(1, -1)

## 5. 利用上述模块搭建序列分类模型

参考模型结构：Embedding – BiLSTM – Attention – Linear – LogSoftmax

In [342]:
class EncoderRNN(nn.Module):
    def __init__(self, num_vocab, embedding_dim, hidden_size, num_classes):
        """
        参数
            num_vocab: 词表大小
            embedding_dim: 词向量维数
            hidden_size: 隐状态维数
            num_classes: 类别数量
        """
        super(EncoderRNN, self).__init__()
        # TODO
        self.embed = nn.Embedding(num_vocab, embedding_dim) # embedding层
        self.bilstm = BiLSTM(embedding_dim, hidden_size) # 双向LSTM
        self.atten = Attention(2*hidden_size) # attention层
        self.linear = nn.Linear(2*hidden_size, num_classes) # 线性层
        self.logsoftmax = nn.LogSoftmax(dim=-1) # softmax层
    
    def forward(self, x):
        """
        输入
            x: 1 * length, LongTensor
        输出
            outputs
        """
        # TODO
        embeddings = []
        for i in range(len(x)):
            embeddings.append(self.embed(x[i])) 
        embeddings = torch.stack(embeddings, dim=0) # 构建embedding向量
        hiddens = self.bilstm(embeddings) # 双向LSTM模型计算隐藏层
        atten_o = self.atten(hiddens) # 注意力机制
        lin = self.linear(atten_o)
        #print("Linear: ", lin)
        outputs = self.logsoftmax(lin)
        return outputs

## 6. 请利用上述模型在古诗作者分类任务上进行训练和测试

要求选取在验证集上效果最好的模型，输出测试集上的准确率、confusion matrix以及macro-precision/recall/F1，并打印部分测试样例及预测结果。

In [346]:
# TODO
model = EncoderRNN(num_vocab=len(idx2word), embedding_dim=256, hidden_size=256, num_classes=len(idx2label))
model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.5)
criterion = nn.NLLLoss()


def train_loop(model, optimizer, criterion):
    model.train()
    epochloss, curloss = 0, 0
    cnt = 0
    for sample in tqdm(train_data):
        cnt += 1

        # 训练样本
        x, y = make_data(sample[0], sample[1])
        x = x.to(device)
        y = y.to(device)
        output = model(x)

        # 计算loss，回传梯度
        loss = criterion(output, y)
        optimizer.zero_grad()
        loss.backward()

        # 剪裁梯度
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
        
        optimizer.step()
        epochloss += loss.item()
        curloss += loss.item()
        if cnt % 1000 == 0:
            print('cur loss: ', curloss / 1000)
            curloss = 0

    epochloss /= len(train_data)
    return epochloss


def test_loop(model, data):
    model.eval()
    precision = 0.0
    pred = []
    for sample in tqdm(data):
        x, y = make_data(sample[0], sample[1])
        x = x.to(device)
        y = y.to(device)
        output = model(x).squeeze(0)

        # 验证准确率
        if torch.argmax(output) == y.item():
            precision += 1
        pred.append(idx2label[torch.argmax(output)])
    precision /= len(data)
    return pred, precision


In [348]:
model.load_state_dict(torch.load("model_best.pt"))

best_score = 0.0
# 模型训练
for epoch in range(5):
    loss = train_loop(model, optimizer, criterion)
    print("epoch loss: ", loss)
    pred, precision = test_loop(model, valid_data)
    if precision > best_score:
        torch.save(model.state_dict(), "model_best.pt")
        best_score = precision
    print("pred: ", precision)
    for i in range(10):
        print(valid_data[i][0], valid_data[i][1], pred[i])

print("Best score: ", best_score)

# 模型预测
model.load_state_dict(torch.load("model_best.pt"))
pred, precision = test_loop(model, test_data)
print("test score: ", precision)
for i in range(10):
    print(test_data[i][0], test_data[i][1], pred[i])

  9%|▉         | 1002/11271 [01:19<11:48, 14.49it/s]

cur loss:  1.5445391139080475


 18%|█▊        | 2001/11271 [02:35<11:24, 13.54it/s]

cur loss:  1.5589832499754694


 27%|██▋       | 3003/11271 [03:58<09:58, 13.80it/s]

...

epoch loss:  0.23505649025438283


100%|██████████| 1408/1408 [01:07<00:00, 20.99it/s]


pred:  0.5447443181818182
我醉君複樂，陶然共忘機。 李白 李白
遠聞房太守，歸葬陸渾山。一德興王后，孤魂久客間。 杜甫 杜甫
西涉清洛源，頗驚人世喧。采秀臥王屋，因窺洞天門。 李白 李白
郢門一為客，巴月三成弦。朔風正搖落，行子愁歸旋。 李白 李白
每遇登臨好風景，羨他天性少情人。 劉禹錫 劉禹錫
檀槽一抹廣陵春，定子初開睡臉新。 李商隱 杜牧
出郭眄細岑，披榛得微路。溪行一流水，曲折方屢渡。 杜甫 杜甫
少年入內教歌舞，不識君王到老時。 杜牧 杜牧
洪壚作高山，元氣鼓其橐。俄然神功就，峻拔在寥廓。 劉禹錫 杜甫
眥血下沾襟，天高問無期。卻尋故鄉路，孤影空相隨。 劉禹錫 杜甫
Best score:  0.5504261363636364


100%|██████████| 1410/1410 [01:05<00:00, 21.52it/s]

test score:  0.5589488636363636
舊日重陽日，傳杯不放杯。即今蓬鬢改，但愧菊花開。 杜甫 杜甫
熊羆交黑槊，賓客滿青油。今日文章主，梁王不姓劉。 劉禹錫 杜甫
晝號夜哭兼幽顯，早晚星關雪涕收。 李商隱 杜甫
玉壘高桐拂玉繩，上含非霧下含冰。 李商隱 李商隱
相思樹上合歡枝，紫鳳青鸞共羽儀。 李商隱 劉禹錫
空齋寂寂不生塵，藥物方書繞病身。纖草數莖勝靜地， 劉禹錫 劉禹錫
陰騭今如此，天災未可無。莫憑牲玉請，便望救焦枯。 李商隱 李白
露索秦宮井，風弦漢殿箏。幾時綿竹頌，擬薦子虛名。 李商隱 杜甫
開從綠條上，散逐香風遠。故取花落時，悠揚占春晚。 劉禹錫 劉禹錫
顧于韓蔡內，辨眼工小字。分日示諸王，鉤深法更秘。 杜甫 杜甫


In [350]:
model2 = EncoderRNN(num_vocab=len(idx2word), embedding_dim=256, hidden_size=256, num_classes=len(idx2label))
model2.to(device)
model2.load_state_dict(torch.load("model_best.pt"))

best_score2 = best_score
for epoch in range(2):
    loss = train_loop(model, optimizer, criterion)
    print("epoch loss: ", loss)
    pred, precision = test_loop(model, valid_data)
    if precision > best_score2:
        torch.save(model.state_dict(), "model_best2.pt")
        best_score2 = precision
    print("pred: ", precision)
    for i in range(10):
        print(valid_data[i][0], valid_data[i][1], pred[i])

print("Best score: ", best_score2)

model2.load_state_dict(torch.load("model_best2.pt"))
pred2, precision2 = test_loop(model, test_data)
print("test score: ", precision2)
for i in range(10):
    print(test_data[i][0], test_data[i][1], pred2[i])


  9%|▉         | 1002/11271 [01:10<11:03, 15.47it/s]

cur loss:  0.2926972826416932


 18%|█▊        | 2003/11271 [02:20<10:38, 14.51it/s]

cur loss:  0.26446440713433517


 27%|██▋       | 3003/11271 [03:31<09:29, 14.51it/s]

...

epoch loss:  0.12440281718934729


100%|██████████| 1408/1408 [00:53<00:00, 26.40it/s]


pred:  0.5596590909090909
我醉君複樂，陶然共忘機。 李白 李白
遠聞房太守，歸葬陸渾山。一德興王后，孤魂久客間。 杜甫 杜甫
西涉清洛源，頗驚人世喧。采秀臥王屋，因窺洞天門。 李白 李白
郢門一為客，巴月三成弦。朔風正搖落，行子愁歸旋。 李白 劉禹錫
每遇登臨好風景，羨他天性少情人。 劉禹錫 劉禹錫
檀槽一抹廣陵春，定子初開睡臉新。 李商隱 杜牧
出郭眄細岑，披榛得微路。溪行一流水，曲折方屢渡。 杜甫 杜甫
少年入內教歌舞，不識君王到老時。 杜牧 杜牧
洪壚作高山，元氣鼓其橐。俄然神功就，峻拔在寥廓。 劉禹錫 杜甫
眥血下沾襟，天高問無期。卻尋故鄉路，孤影空相隨。 劉禹錫 杜甫
Best score:  0.5596590909090909


100%|██████████| 1410/1410 [00:54<00:00, 25.91it/s]

test score:  0.5745738636363636
舊日重陽日，傳杯不放杯。即今蓬鬢改，但愧菊花開。 杜甫 杜甫
熊羆交黑槊，賓客滿青油。今日文章主，梁王不姓劉。 劉禹錫 杜甫
晝號夜哭兼幽顯，早晚星關雪涕收。 李商隱 杜甫
玉壘高桐拂玉繩，上含非霧下含冰。 李商隱 李商隱
相思樹上合歡枝，紫鳳青鸞共羽儀。 李商隱 李白
空齋寂寂不生塵，藥物方書繞病身。纖草數莖勝靜地， 劉禹錫 劉禹錫
陰騭今如此，天災未可無。莫憑牲玉請，便望救焦枯。 李商隱 杜牧
露索秦宮井，風弦漢殿箏。幾時綿竹頌，擬薦子虛名。 李商隱 杜甫
開從綠條上，散逐香風遠。故取花落時，悠揚占春晚。 劉禹錫 劉禹錫
顧于韓蔡內，辨眼工小字。分日示諸王，鉤深法更秘。 杜甫 杜甫


In [352]:
# 测试
best_model = EncoderRNN(num_vocab=len(idx2word), embedding_dim=256, hidden_size=256, num_classes=len(idx2label))
best_model.to(device)
best_model.load_state_dict(torch.load("./model_best2.pt"))
predict, accuracy = test_loop(best_model, test_data)

100%|██████████| 1410/1410 [00:43<00:00, 32.23it/s]


In [360]:
# 预测指标
from sklearn.metrics import confusion_matrix, f1_score, recall_score, accuracy_score
y_pred = [label2idx[author] for author in predict]
y_true = [label2idx[author] for (poem, author) in test_data]
print("Accuracy: ", accuracy_score(y_true, y_pred))
print("Confusion matrix: \n", confusion_matrix(y_true, y_pred))
print("Recall: ", recall_score(y_true, y_pred, average='micro'))
print("F1 score: ", f1_score(y_true, y_pred, average='micro'))

Accuracy:  0.573758865248227
Confusion matrix: 
 [[ 39  28  53  25  15]
 [ 15 299  47  36  17]
 [ 23  61 322  44  18]
 [ 18  38  47 115  19]
 [ 20  22  34  21  34]]
Recall:  0.573758865248227
F1 score:  0.573758865248227


In [363]:
# 样本测试，第一列是诗句，第二列是正确的作者，第三列是预测的作者
for i in range(20):
    print(test_data[i][0], test_data[i][1], predict[i])

舊日重陽日，傳杯不放杯。即今蓬鬢改，但愧菊花開。 杜甫 杜甫
熊羆交黑槊，賓客滿青油。今日文章主，梁王不姓劉。 劉禹錫 杜甫
晝號夜哭兼幽顯，早晚星關雪涕收。 李商隱 杜甫
玉壘高桐拂玉繩，上含非霧下含冰。 李商隱 李商隱
相思樹上合歡枝，紫鳳青鸞共羽儀。 李商隱 李白
空齋寂寂不生塵，藥物方書繞病身。纖草數莖勝靜地， 劉禹錫 劉禹錫
陰騭今如此，天災未可無。莫憑牲玉請，便望救焦枯。 李商隱 杜牧
露索秦宮井，風弦漢殿箏。幾時綿竹頌，擬薦子虛名。 李商隱 杜甫
開從綠條上，散逐香風遠。故取花落時，悠揚占春晚。 劉禹錫 劉禹錫
顧于韓蔡內，辨眼工小字。分日示諸王，鉤深法更秘。 杜甫 杜甫
貧家羞好客，語拙覺辭繁。三朝空錯莫，對飯卻慚冤。 李白 杜甫
吾愛王子晉，得道伊洛濱。金骨既不毀，玉顏長自春。 李白 李白
開元皇帝東封時，百神受職爭賓士。千鈞猛簴順流下， 劉禹錫 杜甫
微雨秋栽竹，孤燈夜讀書。憐君亦同志，晚歲傍山居。 杜牧 杜牧
蘆白疑粘鬢，楓丹欲照心。歸期無雁報，旅抱有猿侵。 李商隱 李白
烈士擊玉壺，壯心惜暮年。三杯拂劍舞秋月， 李白 李白
江色綠且明，茫茫與天平。逶迤巴山盡，搖曳楚雲行。 李白 李白
豈思鱗作簟，仍計腹為燈。浩蕩天池路，翱翔欲化鵬。 李商隱 杜甫
黃衫年少來宜數，不見堂前東逝波。 杜甫 杜甫
繁弦迸關紐，塞管裂圓蘆。眾音不能逐，嫋嫋穿雲衢。 杜牧 杜甫
